<a href="https://colab.research.google.com/github/kushagra6112/IMDB-Sentiment-Analysis/blob/main/IMDB_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

Import Dependencies

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


DATA COLLECTION USING KAGGLE API

In [6]:
kaggle_dictionary=json.load(open("/content/kaggle (1).json"))
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
#unzip the data set file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 43.0MB/s]
100% 25.7M/25.7M [00:00<00:00, 131MB/s] 


**Loading the data set**

In [7]:
data=pd.read_csv("/content/IMDB Dataset.csv")


In [8]:
print("The first few entries")
data.head()


The first few entries


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
print("The last few entries")
data.tail()

The last few entries


,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [10]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

Splitting Data into Training and Test Data Sets

In [11]:
# split data into training data and test data
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


DATA PRE-PROCESSING

In [12]:
##Tokenize text data
tokenizer =Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
#pad_sequences is done to make the same length of the data
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [13]:
#Converted into data
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [14]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

In [15]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - Long Short-Term Memory

In [16]:
## build the model
model= Sequential()
#5000 comes from tokenizer(num_words)
model.add(Embedding(5000,output_dim=128))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))


In [17]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [18]:
##Training the model
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 132s 250ms/step - accuracy: 0.7282 - loss: 0.5243 - val_accuracy: 0.8266 - val_loss: 0.3936
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 245ms/step - accuracy: 0.8468 - loss: 0.3632 - val_accuracy: 0.8347 - val_loss: 0.4023
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 247ms/step - accuracy: 0.8650 - loss: 0.3313 - val_accuracy: 0.8602 - val_loss: 0.3585
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 160s 284ms/step - accuracy: 0.8904 - loss: 0.2714 - val_accuracy: 0.8460 - val_loss: 0.4092
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 270ms/step - accuracy: 0.8989 - loss: 0.2534 - val_accuracy: 0.8685 - val_loss: 0.3322


Model Evaluation

In [19]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"Test loss:{loss}")
print(f"Test accuracy:{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - accuracy: 0.8698 - loss: 0.3240
Test loss:0.32170379161834717
Test accuracy:0.8680999875068665


Building a Predictive System

In [34]:
from collections.abc import Sequence
def predict_sentiment(review):
  # tokenize and pad the review text
  sequence=tokenizer.texts_to_sequences([review]);
  padded_sequence=pad_sequences(sequence,maxlen=200)
  #predict is of the form [[x,y]]-x is for positive and y is for negative
  #x+y=1
  prediction=model.predict(padded_sequence)
  sentiment= "positive" if prediction[0][0]>0.5 else "negative"
  return sentiment


In [30]:
review1="Good movie"
print(predict_sentiment(review1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
positive


In [31]:
review2="The movie is a complete waste of time"
print(predict_sentiment(review2))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
negative


In [35]:
review3="The actors and the team have  done a great job"
print(predict_sentiment(review3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
positive


In [33]:
review4="Helped in distressing,recommend to everyone"
print(predict_sentiment(review4))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
positive


In [36]:
review5="The direction needs a lot of betterment"
print(predict_sentiment(review5))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
negative
